<a href="https://colab.research.google.com/github/FrancescoTorella/progettoLabAI/blob/main/testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install rasterio
import torch
import torchvision
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms.functional as TF
import os
import rasterio
from PIL import Image
from torch.utils.data import Dataset
import numpy as np
from torch.utils.data import DataLoader
import shutil
import matplotlib.pyplot as plt
import time
import random
from sklearn.metrics import precision_score, recall_score, f1_score
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 51.8 MB/s eta 0:00:00
Mounted at /content/drive


In [3]:
!cp -r drive/MyDrive/progettoLabAi3/train/PS-RGBNIR/test_set /content/images

In [4]:
!cp -r drive/MyDrive/progettoLabAi3/train/mask/test_set /content/mask

In [5]:
class MultispectralDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, self.images[index])
        string = self.images[index]
        mask_name = string.replace('PS-RGBNIR','Buildings').replace(".tif","_mask.tiff")
        #print("STRINGA"+ string)
        #tile_index= indice_numero = max(string.rfind('_'), string.rfind(' '))
        #mask_name = string[:tile_index + 1] + "mask_" + string[tile_index + 1:]
        #print("FINALE: "+ mask_name)
        #mask_name = mask_name.replace("11 mask_2.tiff","mask_11.tiff")
        #mask_name = mask_name.replace("12 mask_2.tiff","mask_12.tiff")
        #mask_name = mask_name.replace("13 mask_2.tiff","mask_23.tiff")
        #mask_name = mask_name.replace("21 mask_2.tiff","mask_21.tiff")
        #mask_name = mask_name.replace("22 mask_2.tiff","mask_22.tiff")
        #mask_name = mask_name.replace("23 mask_2.tiff","mask_23.tiff")
        #mask_name = mask_name.replace("31 mask_2.tiff","mask_31.tiff")
        #mask_name = mask_name.replace("32 mask_2.tiff","mask_32.tiff")
        #mask_name = mask_name.replace("33 mask_2.tiff","mask_33.tiff")
        mask_path = os.path.join(self.mask_dir, mask_name)
        with rasterio.open(img_path) as src:
            image = src.read().transpose((1, 2, 0)).astype(np.float32)
        with rasterio.open(mask_path) as src:
          mask = src.read().astype(np.float32)

        mask[mask == 255.0] = 1.0
        mask = mask.squeeze(0)

        if self.transform is not None:
            augmentations = self.transform(image=image, mask=mask)
            image = augmentations["image"]
            mask = augmentations["mask"]


        return image, mask

In [6]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)

class UNET(nn.Module):
    def __init__(
            self, in_channels=3, out_channels=1, features=[64, 128, 256, 512],
    ):
        super(UNET, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Down part of UNET
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Up part of UNET
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2,
                )
            )
            self.ups.append(DoubleConv(feature*2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]

            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip)

        return self.final_conv(x)

In [7]:
def load_checkpoint(checkpoint, model):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])

def save_predictions_as_imgs(loader, model, folder="/content/drive/My Drive/progettoLabAi3/computations/saved/images/",
                             device="cuda"):
    model.eval()  # Imposta il modello in modalità di valutazione
    print(f"Saving images to {folder}")
    iteration = 1
    for idx, (x, y) in enumerate(loader):
        print("Iteration: ", iteration)
        iteration += 1
        x = x.to(device=device)
        y = y.to(device=device).unsqueeze(1)
        with torch.no_grad():
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()

        # Concatenare le immagini (maschera originale e predizione) lungo l'asse verticale
        combined = torch.cat((y, preds), dim=2)

        # Salvare l'immagine combinata
        torchvision.utils.save_image(combined, f"{folder}/comparison_{idx}.png")

    model.train()  # Ripristina il modello in modalità di addestramento

from sklearn.metrics import confusion_matrix

def check_accuracy(loader, model, loss_fn, file, device="cuda"):
    model.eval()
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    precision = 0
    recall = 0
    f1 = 0
    val_loss = 0
    iteration = 1


    all_preds = []
    all_labels = []

    print(f'iterations: {len(loader)}')
    with torch.no_grad():
        for x, y in loader:
            print(f'Iteration number: {iteration}')
            iteration += 1

            x = x.to(device)
            y = y.to(device).unsqueeze(1)

            out = model(x)
            val_loss += loss_fn(out, y).item()
            preds = torch.sigmoid(out)
            preds = (preds > 0.5)
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            dice_score += (2 * (preds * y).sum()) / ((preds + y).sum() + 1e-8)

            # Converti i tensor in numpy array per il calcolo delle metriche
            preds_np = preds.cpu().numpy().flatten()
            y_np = y.cpu().numpy().flatten()

            # Aggiungi le predizioni e le etichette alla lista
            all_preds.extend(preds_np)
            all_labels.extend(y_np)

            precision += precision_score(y_np, preds_np, zero_division=1)
            recall += recall_score(y_np, preds_np, zero_division=1)
            f1 += f1_score(y_np, preds_np, zero_division=1)

    denominator = len(loader)
    val_loss /= denominator
    accuracy = num_correct / num_pixels * 100
    dice = dice_score / denominator
    precision_avg = precision / denominator
    recall_avg = recall / denominator
    f1_avg = f1 / denominator

    # Calcola la matrice di confusione
    cm = confusion_matrix(all_labels, all_preds)
    tn, fp, fn, tp = cm.ravel()

    print(f"Validation Loss: {val_loss:.4f}")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Dice score: {dice}")
    print(f"Precision: {precision_avg}")
    print(f"Recall: {recall_avg}")
    print(f"F1 Score: {f1_avg}")
    print(f"Confusion Matrix:\n {cm}")
    print(f"TN: {tn}, FP: {fp}, FN: {fn}, TP: {tp}")

    # Salva i risultati su un file di testo
    with open(file, "a") as f:
        f.write(f"Validation Loss: {val_loss:.4f}\n")
        f.write(f"Accuracy: {accuracy:.2f}\n")
        f.write(f"Dice score: {dice}\n")
        f.write(f"Precision: {precision_avg}\n")
        f.write(f"Recall: {recall_avg}\n")
        f.write(f"F1 Score: {f1_avg}\n")
        f.write(f"Confusion Matrix:\n {cm}\n")
        f.write(f"TN: {tn}, FP: {fp}, FN: {fn}, TP: {tp}\n")
        f.write("\n")  # Riga vuota per separare le epoche

    model.train()


In [10]:
NUM_WORKERS = 4
PIN_MEMORY = True
IMAGE_HEIGHT = 300
IMAGE_WIDTH = 300
TEST_IMG_DIR = "/content/images/test_set"
TEST_MASK_DIR =  "/content/mask/test_set"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 16


#trasformazioni
test_transform = A.Compose(
      [
          A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
          A.Normalize(
              mean=[0.0,0.0,0.0,0.0],
              std=[1.0,1.0,1.0,1.0],
          ),
          ToTensorV2(),
      ],
  )





def main():

    #lista di directory dei modelli
    model_paths = [
        #"/content/drive/My Drive/progettoLabAi3/computations/saved/model14/",
        #"/content/drive/My Drive/progettoLabAi3/computations/saved/model11/",
        #"/content/drive/My Drive/progettoLabAi3/computations/saved/model4/",
        "/content/drive/My Drive/progettoLabAi3/computations/saved/model12/"
    ]

    models = []

    test_ds = MultispectralDataset(
            image_dir= TEST_IMG_DIR,
            mask_dir= TEST_MASK_DIR,
            transform= test_transform,
    )


    test_loader = DataLoader(
            test_ds,
            batch_size=BATCH_SIZE,
            num_workers= NUM_WORKERS,
            pin_memory=PIN_MEMORY,
            shuffle=False,
    )

    loss_fn = nn.BCEWithLogitsLoss()

    for model_path in model_paths:

        model = UNET(in_channels=4, out_channels=1).to(DEVICE)
        load_checkpoint(torch.load(model_path +"my_checkpoint.pth.tar"), model)
        models.append(model)



    for i in range(0,len(models)):

      model = models[i]
      model_path = model_paths[i]
      print(f"Sto testando il modello {model_path}")
      check_accuracy(
            test_loader,
            model,
            loss_fn,
            file= model_path  +"outputs/test.txt",
            device=DEVICE
      )

      save_predictions_as_imgs(
              test_loader, model, folder=model_path +"images_test/", device=DEVICE
      )


if __name__ == "__main__":
  main()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


=> Loading checkpoint
Sto testando il modello /content/drive/My Drive/progettoLabAi3/computations/saved/model12/
iterations: 32
Iteration number: 1
Iteration number: 2
Iteration number: 3
Iteration number: 4
Iteration number: 5
Iteration number: 6
Iteration number: 7
Iteration number: 8
Iteration number: 9
Iteration number: 10
Iteration number: 11
Iteration number: 12
Iteration number: 13
Iteration number: 14
Iteration number: 15
Iteration number: 16
Iteration number: 17
Iteration number: 18
Iteration number: 19
Iteration number: 20
Iteration number: 21
Iteration number: 22
Iteration number: 23
Iteration number: 24
Iteration number: 25
Iteration number: 26
Iteration number: 27
Iteration number: 28
Iteration number: 29
Iteration number: 30
Iteration number: 31
Iteration number: 32
Validation Loss: 0.0549
Accuracy: 97.80
Dice score: 0.8479240536689758
Precision: 0.8801447086464811
Recall: 0.8181962437589823
F1 Score: 0.8479241423044126
Confusion Matrix:
 [[42146109   384280]
 [  626179  

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Iteration:  1
Iteration:  2
Iteration:  3
Iteration:  4
Iteration:  5
Iteration:  6
Iteration:  7
Iteration:  8
Iteration:  9
Iteration:  10
Iteration:  11
Iteration:  12
Iteration:  13
Iteration:  14
Iteration:  15
Iteration:  16
Iteration:  17
Iteration:  18
Iteration:  19
Iteration:  20
Iteration:  21
Iteration:  22
Iteration:  23
Iteration:  24
Iteration:  25
Iteration:  26
Iteration:  27
Iteration:  28
Iteration:  29
Iteration:  30
Iteration:  31
Iteration:  32
